# Load data

In [ ]:
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
with open('../source/output/results_exact.json') as data_file:    
    data_exact = json.load(data_file)
with open('../source/output/results_heuristic.json') as data_file:    
    data_heuristic = json.load(data_file)
with open('../source/output/results_fast.json') as data_file:    
    data_fast = json.load(data_file)

In [ ]:
def bucket_list(lens, vals):
    vals_list = [list() for i in range(int(max(lens)/5)+1)]
    for i in range(len(lens)):
        vals_list[int(lens[i]/5)].append(vals[i])
    for i in range(len(vals_list)):
        vals_list[i] = np.array(vals_list[i])
    return vals_list

# Exact results

In [ ]:
results_exact = list()
for node in data_exact['nodes']:
    for result in node['tag_results']:
        result['configuration']['id'] = node['id']
        results_exact.append(result)

In [ ]:
lens = [float(result['configuration']['minLength'] + result['configuration']['maxLength'])/2000 for result in results_exact]
score = [float(result['output']['scoreBound']) for result in results_exact]
types = ['b' if 'route' in result['output'] else 'r' for result in results_exact]

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Score bounds')
plt.scatter(lens, score, color=types)
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, score), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(14)])
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

In [ ]:
lens = [float(result['output']['route']['tags']['length'])/1000 for result in results_exact if 'route' in result['output']]
weights = [float(result['output']['route']['tags']['weight']) for result in results_exact if 'route' in result['output']]
interfs = [float(result['output']['route']['tags']['interf']) for result in results_exact if 'route' in result['output']]

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Optimal weights')
plt.scatter(lens, weights)
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, weights), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(14)])
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Optimal interference')
plt.scatter(lens, interfs)
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, interfs), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(14)])
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

# Approximation algorithm (approx)

## Best Scores

In [ ]:
results_heuristic = list()
for node in data_heuristic['nodes']:
    for result in node['tag_results']:
        result['configuration']['id'] = node['id']
        results_heuristic.append(result)

In [ ]:
for result in results_heuristic:
    if len(result['output']['paths']):
        result['output']['bestOutput'] = min(result['output']['paths'], key=lambda p: p['tags']['score'])

In [ ]:
lens = [float(result['output']['bestOutput']['tags']['length'])/1000 for result in results_heuristic if 'bestOutput' in result['output']]
score = [float(result['output']['bestOutput']['tags']['score']) for result in results_heuristic if 'bestOutput' in result['output']]
weights = [float(result['output']['bestOutput']['tags']['weight']) for result in results_heuristic if 'bestOutput' in result['output']]
interfs = [float(result['output']['bestOutput']['tags']['interf']) for result in results_heuristic if 'bestOutput' in result['output']]

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Best scores (4 attempts)')
plt.scatter(lens, score)
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, score), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(14)])
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Best weights (4 attempts)')
plt.scatter(lens, weights)
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, weights), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(14)])
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Best interference (4 attempts)')
plt.scatter(lens, interfs)
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, interfs), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(14)])
plt.ylim(ymin=0)
plt.xlim([0, 70])
plt.show()

In [ ]:
idLen_to_exOut = { str(result['configuration']['id']) + '_' + str(result['configuration']['minLength']): result['output'] for result in results_exact}
for result in results_heuristic:
    key = str(result['configuration']['id']) + '_' + str(result['configuration']['minLength'])
    result['exactOutput'] = idLen_to_exOut[key]

In [ ]:
lens = [ (result['configuration']['minLength'] + result['configuration']['maxLength'])/2000 for result in results_heuristic if 'bestOutput' in result['output']]
deltas = [ result['output']['bestOutput']['tags']['score']-result['exactOutput']['scoreBound'] for result in results_heuristic if 'bestOutput' in result['output']]

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Best deltas (4 attempts)')
plt.scatter(lens, deltas)
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, deltas), positions = [2.5+i*5 for i in range(14)], whis=[5, 95], sym='')
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

## All scores

In [ ]:
paths = list()
for node in data_heuristic['nodes']:
    for result in node['tag_results']:
        for path in result['output']['paths']:
            path['id'] = node['id']
            path['minLength'] = result['configuration']['minLength']
            path['maxLength'] = result['configuration']['maxLength']
            path['id'] = node['id']
            path['reach'] = result['configuration']['reach']
            paths.append(path)

In [ ]:
lens = [float(path['tags']['length'])/1000 for path in paths]
score = [float(path['tags']['score']) for path in paths]
weights = [float(path['tags']['weight']) for path in paths]
interfs = [float(path['tags']['interf']) for path in paths]

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Path scores')
plt.scatter(lens, score)
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, score), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(14)])
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Path weights')
plt.scatter(lens, weights)
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, weights), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(14)])
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Path interferences')
plt.scatter(lens, interfs)
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, interfs), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(14)])
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

In [ ]:
for path in paths:
    key = str(path['id']) + '_' + str(path['minLength'])
    path['exactOutput'] = idLen_to_exOut[key]

In [ ]:
lens = [ (path['minLength'] + path['maxLength'])/2000 for path in paths]
deltas = [ path['tags']['score'] - path['exactOutput']['scoreBound'] for path in paths]

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Path deltas')
plt.scatter(lens, deltas)
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, deltas), positions = [2.5+i*5 for i in range(14)], whis=[5, 95], sym='')
plt.xlim([0, 70])
plt.ylim(ymin=0)
plt.show()

# Approximation algorithm (speed)

In [ ]:
max_len_km = 150
categories = np.array([x for x in  range(2500, max_len_km*1000+2500, 5000)])
reaches = np.array([x for x in range(0, 1500, 500)])

## Best scores

In [ ]:
results_fast = list()
for node in data_fast['nodes']:
    for result in node['tag_results']:
        result['configuration']['id'] = node['id']
        results_fast.append(result)

In [ ]:
for result in results_fast:
    if len(result['output']['paths']):
        result['output']['bestOutput'] = min(result['output']['paths'], key=lambda p: p['tags']['score'])

In [ ]:
lens = [float(result['output']['bestOutput']['tags']['length'])/1000 for result in results_fast if 'bestOutput' in result['output']]
score = [float(result['output']['bestOutput']['tags']['score']) for result in results_fast if 'bestOutput' in result['output']]
weights = [float(result['output']['bestOutput']['tags']['weight']) for result in results_fast if 'bestOutput' in result['output']]
interfs = [float(result['output']['bestOutput']['tags']['interf']) for result in results_fast if 'bestOutput' in result['output']]

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Best scores (4 attempts)')
plt.scatter(lens, score)
plt.xlim([0, max_len_km])
plt.ylim(ymin=0)
plt.ylabel('Score')
plt.xlabel('Tour length (km)')

plt.show()
plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, score), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(30)])
plt.ylim(ymin=0)
plt.xlim([0, 70])
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Best weights (4 attempts)')
plt.scatter(lens, weights)
plt.xlim([0, max_len_km])
plt.ylim(ymin=0)
plt.ylabel('Score')
plt.xlabel('Tour length (km)')
plt.show()

plt.show()
plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, weights), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(30)])
plt.ylim(ymin=0)
plt.xlim([0, 70])
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Best interferences (4 attempts)')
plt.scatter(lens, interfs)
plt.xlim([0, max_len_km])
plt.ylim(ymin=0)
plt.ylabel('Score')
plt.xlabel('Tour length (km)')
plt.show()

plt.show()
plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, interfs), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(30)])
plt.ylim(ymin=0)
plt.xlim([0, 70])
plt.show()

In [ ]:
outliers = list()
confs = 0
interf_confs = 0
for node in data_fast['nodes']:
    ls = list()
    for result in node['tag_results']:
        confs += 1
        if 'bestOutput' in result['output']:
            if float(result['output']['bestOutput']['tags']['score'])>1.5 and result['configuration']['minLength']>40000:
                ls.append(str(result['configuration']['minLength'])+'-'+str(result['configuration']['maxLength']))
                interf_confs += 1
    if len(ls):
        outliers.append({'id': node['id'], 'lat': node['lat'], 'lon': node['lon'], 'problems': str(ls).replace('\'', '')})
        
with open('best_outliers.json', 'w') as data_file:
    json.dump({'nodes': outliers}, data_file)
    
print("Outlier nodes: " + str(100*float(len(outliers))/len(data_fast['nodes'])) + '%')
print("High interference configurations: " + str(100*float(interf_confs)/confs) + '%')

In [ ]:
outliers = list()
confs = 0
unachieved_confs = 0
unachieved_rates = list()
for node in data_fast['nodes']:
    ls = list()
    node_confs = 0
    node_unachieved_confs = 0
    for result in node['tag_results']:
        node_confs += 1
        if not 'bestOutput' in result['output']:
            node_unachieved_confs += 1
            ls.append(str(result['configuration']['minLength'])+'-'+str(result['configuration']['maxLength']))
    confs += node_confs
    unachieved_confs += node_unachieved_confs
    unachieved_rate = float(node_unachieved_confs)/node_confs
    unachieved_rates.append(unachieved_rate)
    outliers.append({'id': node['id'], 'lat': node['lat'], 'lon': node['lon'], 'unachieved_rate': unachieved_rate, 'problems': str(ls).replace('\'', '')})
        
with open('best_unachieved.json', 'w') as data_file:
    json.dump({'nodes': outliers}, data_file)
    
print("Failed configurations: " + str(100*float(unachieved_confs)/confs) + '%')

In [ ]:
plt.hist(unachieved_rates, range=[0, 0.2])
plt.title('Node configuration failure rate')
plt.xlabel('Failure rate (%)')
plt.ylabel('# nodes')
plt.show()

## All scores

In [ ]:
paths = list()
for node in data_fast['nodes']:
    for result in node['tag_results']:
        for path in result['output']['paths']:
            path['id'] = node['id']
            path['minLength'] = result['configuration']['minLength']
            path['maxLength'] = result['configuration']['maxLength']
            path['id'] = node['id']
            path['reach'] = result['configuration']['reach']
            paths.append(path)

In [ ]:
lens = [float(path['tags']['length'])/1000 for path in paths]
score = [float(path['tags']['score']) for path in paths]
weights = [float(path['tags']['weight']) for path in paths]
interfs = [float(path['tags']['interf']) for path in paths]

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Path scores')
plt.scatter(lens, score)
plt.xlim([0, max_len_km])
plt.ylim(ymin=0)
plt.ylabel('Score')
plt.xlabel('Tour length (km)')

plt.show()
plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, score), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(30)])
plt.ylim(ymin=0)
plt.xlim([0, 70])
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Path weights')
plt.scatter(lens, weights)
plt.xlim([0, max_len_km])
plt.ylim(ymin=0)
plt.ylabel('Score')
plt.xlabel('Tour length (km)')
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, weights), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(30)])
plt.ylim(ymin=0)
plt.xlim([0, 70])
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Path interferences')
plt.scatter(lens, interfs)
plt.xlim([0, max_len_km])
plt.ylim(ymin=0)
plt.ylabel('Score')
plt.xlabel('Tour length (km)')
plt.show()

plt.figure(figsize=(13, 8))
plt.boxplot(bucket_list(lens, interfs), whis=[5, 95], sym='', positions = [2.5+i*5 for i in range(30)])
plt.ylim(ymin=0)
plt.xlim([0, 70])
plt.show()

In [ ]:
outliers = list()
confs = 0
unachieved_confs = 0
unachieved_rates = list()
outliers = list()
for node in data_fast['nodes']:
    node_confs = 0
    node_unachieved_confs = 0
    for result in node['tag_results']:
        node_confs += result['configuration']['alternatives']
        node_unachieved_confs += result['configuration']['alternatives'] - len(result['output']['paths'])
            
    confs += node_confs
    unachieved_confs += node_unachieved_confs
    unachieved_rate = float(node_unachieved_confs)/node_confs
    unachieved_rates.append(unachieved_rate)
    outliers.append({'id': node['id'], 'lat': node['lat'], 'lon': node['lon'], 'unachieved_rate': unachieved_rate})
        
with open('path_unachieved.json', 'w') as data_file:
    json.dump({'nodes': outliers}, data_file)

print("Failed attempts: " + str(100*float(unachieved_confs)/confs) + '%')

In [ ]:
plt.hist(unachieved_rates, range=[0, 0.2])
plt.title('Node attempt failure rate')
plt.xlabel('Failure rate (%)')
plt.ylabel('# nodes')
plt.show()

## Execution time

### Outliers

In [ ]:
extract = list()
forward = list()
backward = list()
rem = list()
lengths = list()

for result in results_fast:
    extract_time = result['output']['extractTime']
    forward_time = result['output']['forwardTime']
    avg_backward_time = result['output']['backwardTimeAvg']
    attempts = result['configuration']['alternatives']
    avg_rem_time = (result['output']['time']-attempts*avg_backward_time-forward_time-extract_time)/attempts
    reach_ind = int(result['configuration']['reach']/500)
    extract.append(extract_time)
    forward.append(forward_time)
    backward.append(avg_backward_time)
    rem.append(avg_rem_time)
    lengths.append((result['configuration']['minLength']+result['configuration']['maxLength'])/2)
    
extract = np.array(extract)
forward = np.array(forward)
backward = np.array(backward)
rem = np.array(rem)
lengths = np.array(lengths)

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Forward times')
plt.scatter(lengths/1000, forward*1000)
plt.xlim([0, max_len_km])
plt.ylim(ymin=0)
plt.ylabel('Time (ms)')
plt.xlabel('Tour length (km)')
plt.plot([0, 150], [500, 500], 'k-')
plt.show()

plt.figure(figsize=(13, 8))
plt.title('Backward times')
plt.scatter(lengths/1000, backward*1000)
plt.xlim([0, max_len_km])
plt.ylim(ymin=0)
plt.ylabel('Time (ms)')
plt.xlabel('Tour length (km)')
plt.plot([0, 150], [100, 450], 'k-')
plt.show()

In [ ]:
nrOutliers = 0
for result in results_fast:
    outlier = False
    reach_ind = int(result['configuration']['reach']/500)
    length = (result['configuration']['minLength']+result['configuration']['maxLength'])/2
    forward_time = result['output']['forwardTime']
    avg_backward_time = result['output']['backwardTimeAvg']
    if (forward_time>0.5):
        outlier = True
    if (avg_backward_time>0.1+length*(0.45-.1)/150000):
        outlier = True
    result['output']['outlierTime'] = outlier
    nrOutliers += outlier

print('Time outliers: ' + str(100*float(nrOutliers)/len(results_fast)) + '%')

### Average execution times

In [ ]:
extract_per_category = [list() for category in categories]
forward_per_category = [list() for category in categories]
avg_backward_per_category = [list() for category in categories]
avg_rem_per_category = [list() for category in categories]
tot_backward_per_category = [list() for category in categories]
tot_rem_per_category = [list() for category in categories]
extract = list()
forward = list()
backward = list()
rem = list()
lengths = list()

for result in results_fast:
    if not result['output']['outlierTime']:
        extract_time = result['output']['extractTime']
        forward_time = result['output']['forwardTime']
        avg_backward_time = result['output']['backwardTimeAvg']
        attempts = result['configuration']['alternatives']
        avg_rem_time = (result['output']['time']-attempts*avg_backward_time-forward_time-extract_time)/attempts
        category_ind = int(result['configuration']['minLength']/5000)
        extract_per_category[category_ind].append(extract_time)
        forward_per_category[category_ind].append(forward_time)
        avg_backward_per_category[category_ind].append(avg_backward_time)
        avg_rem_per_category[category_ind].append(avg_rem_time)
        extract.append(extract_time)
        forward.append(forward_time)
        backward.append(avg_backward_time)
        rem.append(avg_rem_time)
        lengths.append((result['configuration']['minLength']+result['configuration']['maxLength'])/2)
        tot_backward_per_category[category_ind].append(avg_backward_time*attempts)
        tot_rem_per_category[category_ind].append(avg_rem_time*attempts)
    
    
for i in range(len(categories)):
    extract_per_category[i] = np.array(extract_per_category[i])
    forward_per_category[i] = np.array(forward_per_category[i])
    avg_backward_per_category[i] = np.array(avg_backward_per_category[i])
    avg_rem_per_category[i] = np.array(avg_rem_per_category[i])
    tot_backward_per_category[i] = np.array(tot_backward_per_category[i])
    tot_rem_per_category[i] = np.array(tot_rem_per_category[i])
        
extract = np.array(extract)
forward = np.array(forward)
backward = np.array(backward)
rem = np.array(rem)
lengths = np.array(lengths)

In [ ]:
plt.figure(figsize=(13, 8))
plt.title('Forward times')
plt.scatter(lengths/1000, forward*1000)
plt.xlim([0, max_len_km])
plt.ylim(ymin=0)
plt.ylabel('Time (ms)')
plt.xlabel('Tour length (km)')
plt.plot([0, 150], [500, 500], 'k-')
plt.show()

plt.figure(figsize=(13, 8))
plt.title('Backward times')
plt.scatter(lengths/1000, backward*1000)
plt.xlim([0, max_len_km])
plt.ylim(ymin=0)
plt.ylabel('Time (ms)')
plt.xlabel('Tour length (km)')
plt.plot([0, 150], [100, 450], 'k-')
plt.show()

In [ ]:
avg_extract_per_category = [1000*np.mean(extract_per_category[i]) for i in range(len(categories))]
avg_forward_per_category = [1000*np.mean(forward_per_category[i]) for i in range(len(categories))]
avg_avg_backward_per_category = [1000*np.mean(avg_backward_per_category[i]) for i in range(len(categories))]
avg_avg_rem_per_category = [1000*np.mean(avg_rem_per_category[i]) for i in range(len(categories))]

In [ ]:
plt.figure()
polys = plt.stackplot(categories/1000, avg_extract_per_category, avg_forward_per_category, avg_avg_backward_per_category, avg_avg_rem_per_category)
legendProxies = []
for poly in polys:
    legendProxies.append(plt.Rectangle((0, 0), 1, 1, fc=poly.get_facecolor()[0]))
plt.legend(legendProxies, ['extraction', 'forward routing', 'backward routing', 'other'], loc='upper left')
plt.title('Single attempt time')
plt.xlabel('Length (km)')
plt.ylabel('Time (ms)')
plt.show()

In [ ]:
plt.figure()
polys = plt.stackplot(categories/1000, avg_forward_per_category, avg_avg_backward_per_category)
legendProxies = []
for poly in polys:
    legendProxies.append(plt.Rectangle((0, 0), 1, 1, fc=poly.get_facecolor()[0]))
plt.legend(legendProxies, ['forward routing', 'backward routing'], loc='upper left')
plt.title('Single attempt time')
plt.xlabel('Length (km)')
plt.ylabel('Time (ms)')
plt.show()

In [ ]:
avg_tot_backward_per_category = [1000*np.mean(tot_backward_per_category[i]) for i in range(len(categories))]
avg_tot_rem_per_category = [1000*np.mean(tot_rem_per_category[i]) for i in range(len(categories))]

In [ ]:
plt.figure()
polys = plt.stackplot(categories/1000, avg_extract_per_category, avg_forward_per_category, avg_tot_backward_per_category, avg_tot_rem_per_category)
legendProxies = []
for poly in polys:
    legendProxies.append(plt.Rectangle((0, 0), 1, 1, fc=poly.get_facecolor()[0]))
plt.legend(legendProxies, ['extraction', 'forward routing', 'backward routing', 'other'], loc='upper left')
plt.title('4 attempts time')
plt.xlabel('Length (km)')
plt.ylabel('Time (ms)')
plt.show()

In [ ]:
plt.figure()
polys = plt.stackplot(categories/1000, avg_forward_per_category, avg_tot_backward_per_category)
legendProxies = []
for poly in polys:
    legendProxies.append(plt.Rectangle((0, 0), 1, 1, fc=poly.get_facecolor()[0]))
plt.legend(legendProxies, ['forward routing', 'backward routing'], loc='upper left')
plt.title('4 attempts time')
plt.xlabel('Length (km)')
plt.ylabel('Time (ms)')
plt.show()